In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
import csv
import numpy as np
import tensorflow as tf
from sklearn import metrics, model_selection

BASE_DIR = '../../../'
import sys
sys.path.append(BASE_DIR)

# custom code
import utils.utils
CONFIG = utils.utils.load_config("../../config.json")

import utils.metrics
import utils.record

Using TensorFlow backend.


In [3]:
DATASET = os.path.basename(os.getcwd()) # name of folder this file is in
RANDOM_SEED = CONFIG['random_seed']
TYPE = CONFIG["experiment_configs"][DATASET]["type"]
NOISE_P = CONFIG["experiment_configs"][DATASET]["noise_p"] # chance of flip
HYPER_VAL_SPLIT = CONFIG["experiment_configs"][DATASET]["hyper_val_split"]

BATCH_SIZE = CONFIG["experiment_configs"][DATASET]["batch_size"]
IMAGE_X = CONFIG["experiment_configs"][DATASET]["image_x_size"]
IMAGE_Y = CONFIG["experiment_configs"][DATASET]["image_y_size"]
IMAGE_SIZE = (IMAGE_Y, IMAGE_X)

print(RANDOM_SEED, TYPE, NOISE_P)

# folders for processed, models
DATA_F = os.path.join(BASE_DIR, f"data/{DATASET}/")
PROCESSED_DIR = os.path.join(BASE_DIR, f'processed/{DATASET}/rs={RANDOM_SEED}')
MODELS_DIR = os.path.join(BASE_DIR, f'models/{DATASET}/rs={RANDOM_SEED}')
RESULTS_DIR = os.path.join(BASE_DIR, 'results')

PROCESSED_SAVEPATH = utils.utils.get_savepath(PROCESSED_DIR, DATASET, ".npz", t=TYPE, np=NOISE_P)
BASE_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt="base", t=TYPE, np=NOISE_P)

# base model saved here
if not os.path.exists(BASE_MODEL_SAVEPATH):
    print(f"warning: no model has been run for rs={RANDOM_SEED}")

15 asym 0.6


In [4]:
os.makedirs(RESULTS_DIR, exist_ok=True)

In [5]:
dat = None
with open(PROCESSED_SAVEPATH, 'rb') as f:
    dat = np.load(f)
    
    ixt = dat['x_train']
    xt = dat['x_hyper_train']

    x_val = dat['x_val']
    y_val = dat['y_val']
    
    x_hyper_val = dat['x_hyper_val']
    y_hyper_val = dat['y_hyper_val']

    x_test = dat['x_test']
    y_test = dat['y_test']

In [6]:
model = utils.utils.make_resnet(
    depth=2,
    random_state=RANDOM_SEED,
    input_shape=(*IMAGE_SIZE, 3),
    nc=10,
)

In [7]:
model_types = [
    "base",
    "ft",
#     "fc",
#     "lrw",
#     "jv"
]

metric_list = [
    'Accuracy',
]

In [8]:
os.makedirs(RESULTS_DIR, exist_ok=True)
savepath = os.path.join(RESULTS_DIR, f"results_{DATASET}.csv")
saver = utils.record.Results_Recorder(savepath, DATASET)

Results file exists, appending to it...


In [9]:
for mt in model_types:
    print(f"Model Type: {mt}")
    modelpath = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt=mt, t=TYPE, np=NOISE_P)
    model.load_weights(modelpath)
    preds_val = utils.utils.compute_preds(
        model,
        x_val,
        batch_size=BATCH_SIZE,
    )
    preds_hyper_val = utils.utils.compute_preds(
        model,
        x_hyper_val,
        batch_size=BATCH_SIZE,
    )
    preds_test = utils.utils.compute_preds(
        model,
        x_test,
        batch_size=BATCH_SIZE,
    )

    preds_val = np.argmax(preds_val, axis=1)
    preds_hyper_val = np.argmax(preds_hyper_val, axis=1)
    preds_test = np.argmax(preds_test, axis=1)

    val_conf = metrics.confusion_matrix(y_val, preds_val, normalize='all')
    hyper_val_conf = metrics.confusion_matrix(y_hyper_val, preds_hyper_val, normalize='all')
    test_conf = metrics.confusion_matrix(y_test, preds_test, normalize='all')
    
    for metric in metric_list:
        val_score = utils.metrics.eval_metric(val_conf, metric)
        hyper_val_score = utils.metrics.eval_metric(hyper_val_conf, metric)
        test_score = utils.metrics.eval_metric(test_conf, metric)
        
        saver.save(RANDOM_SEED, metric, mt, val_score, hyper_val_score, test_score, None)

Model Type: base
Model Type: ft


In [10]:
saver.close()